In [1]:
print("Loading Libraries...")
import os, sys
import platform
import pandas as pd
import dask as dd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
print("Loaded Libraries...")

Loading Libraries...
Loaded Libraries...


In [2]:
#Local
print("Loading directories..")
path_coords = 'D:/01_DOKTORAT/15_results/coords'
path_fft = 'D:/01_DOKTORAT/15_results/fft'
path_fft2 = 'D:/01_DOKTORAT/15_results/fft2'
path_plots = 'D:/01_DOKTORAT/15_results/plots'
print("Loaded directories...")

Loading directories..
Loaded directories...


In [ ]:
filelist = os.listdir(path_fft)
for file in filelist:
    os.chdir(path_fft)
    print(str(file) + ' in processing')
    data = pd.read_csv(file).set_index('nodenumber')
    data = data.applymap(complex)
    mag = data.applymap(lambda x: 2*np.sqrt(x.real**2 + x.imag**2)/5015)
    os.chdir(path_fft2)
    mag.to_csv('mag_' + str(file))
    del mag
    phase = data.applymap(lambda x: np.arctan(x.imag/x.real))
    phase.to_csv('phase_' + str(file))
    del phase
    del data

In [ ]:
#### New stuff here

In [3]:
os.chdir(path_fft2)
filelist = os.listdir(path_fft2)

In [6]:
filelist = filelist[:18]

In [5]:
N = 5015
F = 100000.0
f_step = F/N
f_bin = np.arange(0.0, F, f_step)
f_bin

array([0.00000000e+00, 1.99401795e+01, 3.98803589e+01, ...,
       9.99401795e+04, 9.99601196e+04, 9.99800598e+04])

In [ ]:
max_mag = []

In [ ]:
for file in filelist:
    mag = pd.read_csv(file).set_index('nodenumber')
    mag.columns = np.round(f_bin, decimals=2)
    mag = mag.rename_axis("frequency", axis=1)
    mag = mag.iloc[:,0:1354]
    max_mag.append(mag.max().max())

max_mag

In [ ]:
max(max_mag)

[626.7666901661229,
 696.7789033730122,
 630.192786868897,
 441.6434222302839,
 809.0251337365855,
 1528.1758853159192,
 1263.9725998813851,
 1393.0564298386048,
 1846.2515302153656,
 2599.8225247982496,
 5288.990722794338,
 6676.201234325121,
 1770.1077471703195,
 598.76010841072,
 2805.2703754696254,
 1459.1922057552881,
 1773.723645641225,
 1550.7148668827429]

In [ ]:
for file in filelist:
    mag = pd.read_csv(file).set_index('nodenumber')
    mag.columns = np.round(f_bin, decimals=2)
    mag = mag.rename_axis("frequency", axis=1)
    mag = mag.iloc[:,0:1354]
    mag_log1 = mag.applymap(lambda x: 20*np.log10(x/mag.max().max()))
    mag_log2 = mag.applymap(lambda x: 20*np.log10(x/6676.201234325121))
    
    f, ax = plt.subplots(nrows=4, figsize=(15, 60), dpi=300)
    sns.heatmap(mag, annot=False, linewidths=0, ax=ax[0], cmap=plt.cm.jet, vmin=0, vmax=mag.max().max())
    sns.heatmap(mag, annot=False, linewidths=0, ax=ax[1], cmap=plt.cm.jet, vmin=0, vmax=6676.201234325121)
    sns.heatmap(mag_log1, annot=False, linewidths=0, ax=ax[2], cmap=plt.cm.jet, vmin=-300, vmax=0)
    sns.heatmap(mag_log2, annot=False, linewidths=0, ax=ax[3], cmap=plt.cm.jet, vmin=-300, vmax=0)

In [8]:
mag = pd.read_csv('mag_int-01-fft.csv').set_index('nodenumber')
mag.columns = np.round(f_bin, decimals=2)
mag = mag.rename_axis("frequency", axis=1)
mag = mag.iloc[:,0:1354]
mag_log1 = mag.applymap(lambda x: 20*np.log10(x/mag.max().max()))
mag_log2 = mag.applymap(lambda x: 20*np.log10(x/6676.201234325121))

KeyboardInterrupt: 

In [ ]:
f, ax = plt.subplots(figsize=(15, 15), dpi=300)
sns.heatmap(mag, annot=False, linewidths=0, ax=ax, cmap=plt.cm.jet, vmin=0, vmax=mag.max().max())

In [ ]:
f, ax = plt.subplots(figsize=(15, 15), dpi=300)
sns.heatmap(mag, annot=False, linewidths=0, ax=ax, cmap=plt.cm.jet, vmin=0, vmax=6676.201234325121)

In [ ]:
f, ax = plt.subplots(figsize=(15, 15), dpi=300)
sns.heatmap(mag_log1, annot=False, linewidths=0, ax=ax, cmap=plt.cm.jet, vmin=-500, vmax=0)

In [ ]:
f, ax = plt.subplots(figsize=(15, 15), dpi=300)
sns.heatmap(mag_log2, annot=False, linewidths=0, ax=ax, cmap=plt.cm.jet, vmin=-500, vmax=0)

In [ ]:
X_db = 20*log10(X/max(X));

In [ ]:
f, ax = plt.subplots(figsize=(15, 15), dpi=300)
sns.heatmap(mag_log, annot=False, linewidths=0, ax=ax, cmap=plt.cm.jet)